# Text preprocessing

In [14]:
DATA = "data/"
TRAIN = f"{DATA}/train.csv"
TEST = f"{DATA}/test.csv"
TRAIN_LABEL = f"{DATA}/train_label.txt"
TRAIN_TITLE = f"{DATA}/train_title.txt"
TRAIN_TEXT = f"{DATA}/train_text.txt"
TEST_LABEL = f"{DATA}/test_label.txt"
TEST_TITLE = f"{DATA}/test_title.txt"
TEST_TEXT = f"{DATA}/test_text.txt"

TRAIN_NORM = f"{DATA}/train_norm.txt"
TEST_NORM = f"{DATA}/test_norm.txt"

In [15]:
# # Splt the csv into label, title, text
# import pandas as pd

# train = pd.read_csv(TRAIN)
# train_col = train.columns
# train[train_col[0]].to_csv(TRAIN_LABEL, index=False)
# train[train_col[1]].to_csv(TRAIN_TITLE, index=False)
# train[train_col[2]].to_csv(TRAIN_TEXT, index=False)

# test = pd.read_csv(TEST)
# test_col = test.columns
# test[test_col[0]].to_csv(TEST_LABEL, index=False)
# test[test_col[1]].to_csv(TEST_TITLE, index=False)
# test[test_col[2]].to_csv(TEST_TEXT, index=False)
# del train, test, pd

In [16]:
# from src.Normalizer import normalize_data
# normalize_data(TRAIN_TEXT, TRAIN_NORM)
# normalize_data(TEST_TEXT, TEST_NORM)

In [21]:
import numpy as np
from typing import List, Union, Generator, Dict
import torch
import numpy as np
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
from torch import optim
from collections import Counter
from datetime import time

In [22]:
class Corpus:
	def __init__(self, file_name: str, window: int = 1, k = 3):
		self.file_name = file_name
		self.word2id: Dict[str, int] = {}
		self.id2word: Dict[int, str] = {}
		self.word_count = {}
		self.k = k
		self.vocab_size = 0
		self.window = window
		self.pairs = []
		self.noise_dist = np.array([])
		self.__init_data()
		self.__subsample()
		self.__make_pairs()
		self.__noise_dist()

	def __init_data(self):
		with open(self.file_name, "r") as file:
			data = file.read()
		data = data.split()
		self.word_count = Counter(data)
		self.total = sum(list(self.word_count.values()))
		self.vocab_size = len(self.word_count)

	def __subsample(self):
		t = 1e-5
		sorted_count = sorted(self.word_count, key=self.word_count.get, reverse=True)  # type: ignore
		freq = {word: count / self.total for word, count in self.word_count.items()}
		pdrop = {word: 1 - np.sqrt(t / freq[word]) for word in sorted_count}
		i = 0
		_new_word_count = {}
		for word in sorted_count:
			if np.random.random() < 2:#- pdrop[word]:
				self.id2word[i] = word
				self.word2id[word] = i
				_new_word_count[word] = self.word_count[word]
				i += 1
		self.word_count = _new_word_count
		self.vocab_size =  len(self.word_count.keys())
		self.total = sum(list(self.word_count.values()))

	def __get_pairs(self, text: List[str]):
		num_words = len(text)
		for i, word in enumerate(text):
			if word in self.word2id.keys():
				center = self.word2id[word]
				context = text[max(0, i - self.window) : i]
				context += text[i + 1 : min(num_words, i + self.window + 1)]
				_tmp = ((center, self.word2id[cnt]) for cnt in context if cnt in self.word2id.keys())
				self.pairs.extend(_tmp)  # type: ignore

	def __noise_dist(self,):
		freq = {}
		for word in self.word2id.keys():
			freq[word] = self.word_count[word]/self.total
		unigram = np.array(list(freq.values()))**(3/4)
		self.noise_dist = unigram/unigram.sum()
		self.noise_dist = torch.from_numpy(self.noise_dist)
		_neg = []
		for pair in self.pairs:
			_, b = pair
			_tmp = self.noise_dist.clone()
			_tmp[b] = 0.0
			_neg.append(torch.multinomial(input = _tmp, num_samples = self.k, replacement = True))
		self.neg = np.asarray(_neg)
		i,j = self.pairs.shape
		self.pairs = self.pairs.reshape(i,j,1)

		
	def __make_pairs(self):
		with open(self.file_name) as data:
			for text in data:
				self.__get_pairs(text.strip().split())
		_tmp = set(self.pairs)
		_tmp = np.array(list(_tmp))
		self.pairs = np.array(_tmp)	

In [24]:
corpus = Corpus(TRAIN_NORM, 3, 10)
corpusdata = CorpusData(corpus)
V = corpus.vocab_size
H = 50
device = "mps"

In [25]:
model = SGNS(V, H)
model = model.to(device)

In [26]:
criterion = SGNSLoss()
optimizer = optim.Adam(model.parameters())
dataloader = DataLoader(corpusdata, batch_size=4 * 8192, shuffle=True)
N = len(corpusdata)

In [27]:
for i in range(100):
	log = []
	total_loss = 0
	model.train()
	pbar = tqdm(dataloader, total=len(dataloader), desc=f"Epoch {i+1}")
	for c, o, n in pbar:
		c, o, n = c.to(device), o.to(device), n.to(device)
		pos, neg = model(c, o, n)
		optimizer.zero_grad()
		loss = criterion(pos, neg)
		loss.backward()
		optimizer.step()
		total_loss += loss.item() * c.size(0)
		pbar.set_postfix(loss=f"{total_loss/N:.2f}")

	avg_loss = total_loss / N
	log.append(avg_loss)

Epoch 1:   0%|          | 0/148 [00:00<?, ?it/s]

Epoch 4:   5%|▌         | 8/148 [00:03<01:02,  2.24it/s, loss=1.20]


KeyboardInterrupt: 

In [ ]:
model.vEmbedding.weight.shape

torch.Size([55085, 50])

In [ ]:
from torch.utils.tensorboard.writer import SummaryWriter

from datetime import datetime

In [ ]:
datetime.now()

datetime.datetime(2024, 7, 6, 9, 38, 7, 609074)

In [ ]:
def log_embeddings(model, corpus):
	now = datetime.now()
	log_dir = f'runs/negativesampling_{now.strftime("%Y%m%d-%H%M%S")}'
	writer = SummaryWriter(log_dir)
	embeddings = model.vEmbedding.weight
	labels = list(corpus.id2word.values())
	writer.add_embedding(embeddings, metadata=labels)
	writer.flush()
	writer.close()
	return log_dir


# log_embeddings(model, corpus)

'runs/negativesampling_20240706-093845'